In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_10370/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2_sup")
        # self.discard_below_average(sort_by="mse_sup")
        # self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results_03.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.9)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_3_6,0.979800,0.861657,0.948015,0.941502,0.945567,0.028671,0.196357,0.032905,0.084371,0.058638,0.771553,0.169324,1.000279,0.176533,3525.103757,5669.106333,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
1,model_2_3_5,0.978703,0.861558,0.956181,0.941243,0.947793,0.030228,0.196499,0.027737,0.084745,0.056241,0.795543,0.173861,1.000295,0.181262,3524.998005,5669.000581,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
2,model_2_3_7,0.980528,0.861472,0.940062,0.941312,0.943104,0.027638,0.196620,0.037939,0.084645,0.061292,0.750251,0.166246,1.000269,0.173323,3525.177153,5669.179729,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
3,model_2_3_8,0.980973,0.861081,0.932413,0.940796,0.940511,0.027007,0.197175,0.042781,0.085389,0.064085,0.731306,0.164337,1.000263,0.171333,3525.223338,5669.225914,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
4,model_2_3_4,0.977130,0.861071,0.964422,0.940381,0.949637,0.032461,0.197189,0.022520,0.085988,0.054254,0.822591,0.180170,1.000316,0.187840,3524.855412,5668.857988,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140,model_1_9_18,0.971013,0.824024,0.910892,0.976672,0.953161,0.041142,0.249773,0.042994,0.019120,0.031057,1.058674,0.202836,1.000401,0.211471,3524.381437,5668.384013,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
1141,model_15_6_23,0.955446,0.823916,0.977311,0.706872,0.939775,0.063237,0.249925,0.038651,0.090644,0.064648,1.406112,0.251470,1.008554,0.262176,303.521724,485.134222,"Hidden Size=[20, 4], regularizer=0.5, learning..."
1142,model_1_8_0,0.961507,0.823914,0.980795,0.926354,0.952459,0.054636,0.249928,0.010707,0.073695,0.042201,1.435631,0.233743,1.000532,0.243694,3523.814138,5667.816714,"Hidden Size=[40, 39], regularizer=0.2, learnin..."
1143,model_25_6_0,0.918672,0.823893,0.916428,0.866030,0.907078,0.115433,0.249959,0.079643,0.043514,0.061578,0.803203,0.339755,1.003492,0.354219,1170.318125,1880.922731,"Hidden Size=[26, 18], regularizer=0.2, learnin..."
